
# Text Summarization Using Pre-trained Models
**Goal:** Generate concise summaries from long news articles using pre-trained encoder–decoder models.

**Features**
- Load **CNN/DailyMail** (Kaggle) or your own CSV (`text`, `summary`)
- **Abstractive** summarization with Hugging Face Transformers (BART/T5/Pegasus; picks the first two available)
- Long-text handling: **truncate** to model limits and optional **chunk-and-merge** pipeline
- **ROUGE** evaluation (ROUGE-1/2/L via `rouge-score`)
- **Extractive** summarization with **TextRank** (pure Python: TF-IDF + cosine + PageRank)
- **Fine-tuning** a pre-trained summarizer on a custom dataset (minimal, Trainer API)

**Tools/Libraries:** `transformers`, `pandas`, `torch`, `rouge-score`, `numpy`, `matplotlib`.


## 1) Install dependencies & select models

In [ ]:

# Uncomment as needed in your environment:
# %pip install transformers datasets accelerate rouge-score torch pandas numpy matplotlib nltk PyPDF2 python-docx
# python -m nltk.downloader punkt


## 2) Imports & Setup

In [13]:

import os, json, math, warnings, re
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import sent_tokenize

# Ensure NLTK sentence tokenizer data is available (handles newer nltk punkt_tab)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    pass  # we'll download below

# Prefer a local data directory inside the workspace to avoid permission issues
NLTK_DATA_DIR = os.path.join(os.getcwd(), 'nltk_data')
if NLTK_DATA_DIR not in nltk.data.path:
    nltk.data.path.append(NLTK_DATA_DIR)

for pkg in ("punkt", "punkt_tab"):
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        try:
            nltk.download(pkg, download_dir=NLTK_DATA_DIR, quiet=True)
        except Exception as e:
            warnings.warn(f"Could not download nltk resource '{pkg}': {e}")

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Pick two abstractive models in priority order (will try to load; fallback to next)
CANDIDATE_MODELS = [
    "ainize/bart-base-cnn",
    "google-t5/t5-small"
    # "facebook/bart-large-cnn",
    # "google/pegasus-cnn_dailymail",
]

def try_load_model(name: str):
    try:
        tok = AutoTokenizer.from_pretrained(name)
        mdl = AutoModelForSeq2SeqLM.from_pretrained(name).to(DEVICE)
        return tok, mdl
    except Exception as e:
        warnings.warn(f"Could not load {name}: {e}")
        return None, None

MODELS = []
for m in CANDIDATE_MODELS:
    tok, mdl = try_load_model(m)
    if tok is not None and mdl is not None:
        MODELS.append((m, tok, mdl))
    if len(MODELS) == 2:
        break

if not MODELS:
    raise RuntimeError("No summarization models could be loaded. Install at least one from CANDIDATE_MODELS.")
elif len(MODELS) == 1:
    # Duplicate the only one available for comparison display purposes
    MODELS = MODELS * 2

[m[0] for m in MODELS]


Using device: cpu


['ainize/bart-base-cnn', 'google-t5/t5-small']

## 3) Load Data (CNN/DailyMail)

In [ ]:

def load_dataset() -> pd.DataFrame:
    # Try CSV files
    paths = ["D:/elevvo-mine/datasets/cnn_dailymail/train.csv"]
    for p in paths:
        if os.path.exists(p):
            df = pd.read_csv(p)
            break
        else:
            # Try jsonl
            if os.path.exists("news.jsonl"):
                df = pd.read_json("news.jsonl", lines=True)
            else:
                raise FileNotFoundError("Provide 'news.csv' (text, summary) or 'cnn_dailymail.csv' (article, highlights) or 'news.jsonl'.")

    # Map columns
    col_map = {c.lower(): c for c in df.columns}
    if "text" in col_map:
        text_col = col_map["text"]
    elif "article" in col_map:
        text_col = col_map["article"]
    else:
        raise ValueError("Could not find a text/article column. Add 'text' or 'article'.")

    if "summary" in col_map:
        sum_col = col_map["summary"]
    elif "highlights" in col_map:
        sum_col = col_map["highlights"]
    else:
        sum_col = None  # evaluation will be limited if not provided

    if sum_col is not None:
        df = df[[text_col, sum_col]].rename(columns={text_col:"text", sum_col:"summary"}).dropna()
    else:
        df = df[[text_col]].rename(columns={text_col:"text"}).dropna()

    # Trim whitespace
    df["text"] = df["text"].astype(str).str.strip()
    if "summary" in df.columns:
        df["summary"] = df["summary"].astype(str).str.strip()
    return df

df = load_dataset()
print(df.head())
print("\nRows:", len(df))


                                                text  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                             summary  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...  

Rows: 287113


## 4) Preprocess Long Texts & Truncate/Chunk to Model Limits

In [15]:


def safe_sent_tokenize(text: str) -> List[str]:
    """Try NLTK sent_tokenize; if resources missing, fall back to a simple regex splitter."""
    try:
        return sent_tokenize(text)
    except LookupError:
        # Very lightweight fallback, may not be perfect but avoids crashing
        return [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]


def chunk_text(text: str, tokenizer, max_input_tokens: int = 1024, overlap_sentences: int = 1) -> List[str]:
    """Split long text into sentence chunks that fit the model's input token limit.
    Uses sentence tokenization + greedy packing by tokens.
    """
    sents = safe_sent_tokenize(text)
    chunks, buf = [], []
    # Keep a small sliding overlap of sentences between chunks (if requested)
    for s in sents:
        candidate = " ".join(buf + [s])
        if len(tokenizer.encode(candidate, truncation=False)) <= max_input_tokens:
            buf.append(s)
        else:
            if buf:
                chunks.append(" ".join(buf))
                if overlap_sentences > 0:
                    # take the last k sentences from the previous buffer as overlap
                    buf = buf[-overlap_sentences:]
                else:
                    buf = []
                buf.append(s)
            else:
                # sentence longer than limit: hard truncate tokens
                tokens = tokenizer.encode(s, truncation=True, max_length=max_input_tokens)
                chunks.append(tokenizer.decode(tokens, skip_special_tokens=True))
                buf = []
    if buf:
        chunks.append(" ".join(buf))
    return chunks


def summarize_chunks(chunks: List[str], tokenizer, model, summary_max_tokens=128) -> str:
    partial_summaries = []
    for ch in chunks:
        inputs = tokenizer(
            ch,
            return_tensors="pt",
            truncation=True,
            max_length=tokenizer.model_max_length if tokenizer.model_max_length <= 2048 else 1024
        ).to(DEVICE)
        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=summary_max_tokens,
                num_beams=4,
                length_penalty=1.0,
                early_stopping=True
            )
        partial = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        partial_summaries.append(partial)
    # Merge partial summaries (simple join; you could re-summarize the concatenation)
    merged = " ".join(partial_summaries)
    return merged


def abstractive_summarize(text: str, tokenizer, model, max_input_tokens=1024, summary_max_tokens=128, use_chunking=True) -> str:
    tokens = tokenizer.encode(text, truncation=False)
    if len(tokens) <= max_input_tokens or not use_chunking:
        inputs = tokenizer(
            text, return_tensors="pt", truncation=True, max_length=max_input_tokens
        ).to(DEVICE)
        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=summary_max_tokens,
                num_beams=4,
                length_penalty=1.0,
                early_stopping=True
            )
        return tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    else:
        chunks = chunk_text(text, tokenizer, max_input_tokens=max_input_tokens)
        return summarize_chunks(chunks, tokenizer, model, summary_max_tokens=summary_max_tokens)

# Quick smoke test on one sample
sample_text = df.iloc[0]["text"]
model_name_a, tok_a, mdl_a = MODELS[0]
print("Model A:", model_name_a)
print("Sample summary (first 1000 chars of input):")
print(abstractive_summarize(sample_text[:1000], tok_a, mdl_a, max_input_tokens=1024, summary_max_tokens=64, use_chunking=False))


Model A: ainize/bart-base-cnn
Sample summary (first 1000 chars of input):
Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus .
The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion .
Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus .
The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion .


## 5) Generate Summaries (Compare Two Models)

In [16]:

N_SAMPLES = min(10, len(df))  # smaller for faster rerun; adjust as needed
subset = df.iloc[:N_SAMPLES].copy()

# quick sanity for sentence tokenizer
_ = safe_sent_tokenize(subset.iloc[0]["text"])[:3]

results = []
for (model_name, tok, mdl) in MODELS:
    print(f"\nGenerating with {model_name} ...")
    summaries = []
    for txt in subset["text"].tolist():
        s = abstractive_summarize(txt, tok, mdl, max_input_tokens=1024, summary_max_tokens=128, use_chunking=True)
        summaries.append(s)
    subset[f"summary_{model_name}"] = summaries
    results.append((model_name, summaries))

subset.head()



Generating with ainize/bart-base-cnn ...

Generating with google-t5/t5-small ...

Generating with google-t5/t5-small ...


Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors


,text,summary,summary_ainize/bart-base-cnn,summary_google-t5/t5-small
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",Bishop John Folda of the Fargo Catholic Dioces...,the bishop of the Fargo Catholic Diocese in No...
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,Ralph Mata was an internal affairs lieutenant ...,in the division that investigates allegations ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...","Craig Eccleston-Todd, 27, was texting while dr...","Craig Eccleston-Todd, 27, was driving home fro..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,EU facing urgent calls to widen scope of its m...,"a new chapter into Crimea's turbulent history,..."
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,Fleetwood are the only team still to have a 10...,Fleetwood are the only team to have a 100% rec...


## 6) Evaluate with ROUGE (1/2/L)

In [17]:

def compute_rouge(refs: List[str], hyps: List[str]) -> Dict[str, float]:
    scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
    sums = {"rouge1":0.0,"rouge2":0.0,"rougeL":0.0}
    n = 0
    for r, h in zip(refs, hyps):
        if not isinstance(r, str) or not isinstance(h, str) or not r.strip() or not h.strip():
            continue
        scores = scorer.score(r, h)
        for k in sums.keys():
            sums[k] += scores[k].fmeasure
        n += 1
    return {k: (v/max(n,1)) for k,v in sums.items()}

if "summary" in df.columns:
    refs = subset["summary"].astype(str).tolist()
    score_rows = []
    for (model_name, summaries) in results:
        rouge = compute_rouge(refs, summaries)
        row = {"model": model_name, **{k: round(v,4) for k,v in rouge.items()}}
        score_rows.append(row)
    scores_df = pd.DataFrame(score_rows).sort_values("rougeL", ascending=False)
    print("Average ROUGE F1 scores:")
    display(scores_df)
else:
    print("No reference summaries found — skipping ROUGE. Provide a 'summary' column to evaluate.")


Average ROUGE F1 scores:


,model,rouge1,rouge2,rougeL
0,ainize/bart-base-cnn,0.4113,0.2144,0.3097
1,google-t5/t5-small,0.3460,0.1260,0.2282


## 7) Extractive Summarization (TextRank)

In [18]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


def textrank_summarize(text: str, top_k_sentences: int = 5) -> str:
    sentences = safe_sent_tokenize(text)
    if len(sentences) <= top_k_sentences:
        return text

    # TF-IDF sentence embeddings
    tfidf = TfidfVectorizer(stop_words='english')
    X = tfidf.fit_transform(sentences)
    sim = cosine_similarity(X)

    # Build graph and rank sentences
    np.fill_diagonal(sim, 0.0)
    graph = nx.from_numpy_array(sim)
    scores = nx.pagerank(graph)

    ranked = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)
    # Keep original order for the selected sentences
    selected = sorted(ranked[:top_k_sentences], key=lambda x: x[2])
    summary = " ".join([s for _, s, _ in selected])
    return summary

# Compare extractive vs abstractive on a sample
sample = df.iloc[0]["text"]
extractive = textrank_summarize(sample, top_k_sentences=5)
model_name_b, tok_b, mdl_b = MODELS[0]
abstractive = abstractive_summarize(sample, tok_b, mdl_b, max_input_tokens=1024, summary_max_tokens=128, use_chunking=True)

print("\n--- Extractive (TextRank) ---\n", extractive[:800], "...")
print("\n--- Abstractive ({}) ---\n".format(model_name_b), abstractive[:800], "...")



--- Extractive (TextRank) ---
 14:11 EST, 25 October 2013 . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . Fargo Catholic Diocese in North Dakota (pictured) is where the bishop is located . ...

--- Abstractive (ainize/bart-base-cnn) ---
 Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus .
The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion . ...


## 8) Fine-tune a Pre-trained Summarizer (Trainer API)

In [ ]:

# Tip: Enable GPU and consider gradient accumulation / mixed precision for speed.

from torch.utils.data import Dataset
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments
from dataclasses import fields as dataclass_fields
import inspect
import os

if "summary" not in df.columns:
    print("Fine-tuning skipped: dataset lacks reference summaries. Provide a 'summary' column.")
else:
    FT_MODEL_NAME, FT_TOK, FT_MODEL = MODELS[1]  # choose second model
    print("Fine-tuning base model:", FT_MODEL_NAME)

    # data configuration
    SEED = 42
    FT_MAX_SAMPLES = 200   # cap total examples used for fine-tuning
    FT_TRAIN_FRAC = 0.8    # split within the small subset
    FT_PER_DEVICE_BATCH = 2
    FT_MAX_STEPS = 50     # cap steps for a quick run (if supported)

    # Make a small, reproducible subset with summaries
    df_ft = df.dropna(subset=["text", "summary"]).reset_index(drop=True)
    n_use = min(FT_MAX_SAMPLES, len(df_ft))
    if n_use < 4:
        print(f"Not enough samples to fine-tune (found {len(df_ft)} with summaries). Skipping.")
    else:
        df_small = df_ft.sample(n=n_use, random_state=SEED).reset_index(drop=True)

        class SummarizationDataset(Dataset):
            def __init__(self, df, tokenizer, max_input=1024, max_target=128):
                self.df = df.reset_index(drop=True)
                self.tokenizer = tokenizer
                self.max_input = max_input
                self.max_target = max_target
            def __len__(self): return len(self.df)
            def __getitem__(self, idx):
                row = self.df.iloc[idx]
                inputs = self.tokenizer(
                    row["text"],
                    max_length=self.max_input,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )
                labels = FT_TOK(
                    row["summary"],
                    max_length=self.max_target,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )
                item = {k: v.squeeze(0) for k,v in inputs.items()}
                # Important: replace padding token id's in labels by -100 to ignore loss on padding
                labels_ids = labels["input_ids"].squeeze(0)
                labels_ids[labels_ids == FT_TOK.pad_token_id] = -100
                item["labels"] = labels_ids
                return item

        # Split small subset
        n = len(df_small)
        split = max(1, int(n * FT_TRAIN_FRAC))
        if split >= n:
            split = n - 1  # ensure non-empty eval
        train_df = df_small.iloc[:split].copy()
        eval_df  = df_small.iloc[split:].copy()

        print(f"Training on {len(train_df)} examples; evaluating on {len(eval_df)} examples.")

        train_ds = SummarizationDataset(train_df, FT_TOK)
        eval_ds  = SummarizationDataset(eval_df, FT_TOK)

        collator = DataCollatorForSeq2Seq(tokenizer=FT_TOK, model=FT_MODEL)

        # Build TrainingArguments kwargs compatible with the installed transformers version
        try:
            ta_field_names = {f.name for f in dataclass_fields(TrainingArguments)}
        except Exception:
            ta_field_names = set(inspect.signature(TrainingArguments).parameters.keys())

        def add_if_supported(d, key, value):
            if key in ta_field_names:
                d[key] = value

        ta_kwargs = {}
        add_if_supported(ta_kwargs, "output_dir", "./summarizer_ft")
        add_if_supported(ta_kwargs, "learning_rate", 5e-5)
        add_if_supported(ta_kwargs, "num_train_epochs", 1)
        add_if_supported(ta_kwargs, "weight_decay", 0.01)
        add_if_supported(ta_kwargs, "logging_steps", 10)
        add_if_supported(ta_kwargs, "save_steps", 50)
        add_if_supported(ta_kwargs, "fp16", torch.cuda.is_available())
        add_if_supported(ta_kwargs, "seed", SEED)
        add_if_supported(ta_kwargs, "save_total_limit", 1)

        # Batch size (older versions used per_gpu_* keys)
        if "per_device_train_batch_size" in ta_field_names:
            ta_kwargs["per_device_train_batch_size"] = FT_PER_DEVICE_BATCH
            ta_kwargs["per_device_eval_batch_size"] = FT_PER_DEVICE_BATCH
        else:
            add_if_supported(ta_kwargs, "per_gpu_train_batch_size", FT_PER_DEVICE_BATCH)
            add_if_supported(ta_kwargs, "per_gpu_eval_batch_size", FT_PER_DEVICE_BATCH)

        # Evaluation scheduling
        if "evaluation_strategy" in ta_field_names:
            ta_kwargs["evaluation_strategy"] = "steps"
            add_if_supported(ta_kwargs, "eval_steps", 25)
            add_if_supported(ta_kwargs, "predict_with_generate", True)
        else:
            # Very old versions
            add_if_supported(ta_kwargs, "evaluate_during_training", True)

        # Cap total training steps for a quick run if supported
        add_if_supported(ta_kwargs, "max_steps", FT_MAX_STEPS)

        # Disable external reporters if supported
        add_if_supported(ta_kwargs, "report_to", [])

        args = TrainingArguments(**ta_kwargs)

        trainer = Trainer(
            model=FT_MODEL,
            args=args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            data_collator=collator,
            tokenizer=FT_TOK
        )

        # Train (quick run)
        trainer.train()

        # Save final fine-tuned model in a stable folder for downstream use
        final_dir = os.path.join("summarizer_ft", "final")
        os.makedirs(final_dir, exist_ok=True)
        FT_MODEL.save_pretrained(final_dir)
        FT_TOK.save_pretrained(final_dir)
        print("Saved fine-tuned model to:", os.path.abspath(final_dir))

        # After training, quick evaluation with ROUGE on eval set (first few examples)
        def generate_batch_summaries(texts, tokenizer, model, max_input=1024, max_new=128):
            outs = []
            for t in texts:
                inputs = tokenizer(t, return_tensors="pt", truncation=True, max_length=max_input).to(DEVICE)
                with torch.no_grad():
                    gen_ids = model.generate(**inputs, max_new_tokens=max_new, num_beams=4)
                outs.append(tokenizer.decode(gen_ids[0], skip_special_tokens=True))
            return outs

        sample_evals = eval_df.iloc[:min(20, len(eval_df))]
        if len(sample_evals) > 0:
            preds = generate_batch_summaries(sample_evals["text"].tolist(), FT_TOK, FT_MODEL)
            rouge = compute_rouge(sample_evals["summary"].tolist(), preds)
            print("ROUGE after fine-tuning:", {k: round(v,4) for k,v in rouge.items()})
        else:
            print("No eval samples to score.")


Fine-tuning base model: google-t5/t5-small
Training on 160 examples; evaluating on 40 examples.


C:\Users\Youssef\AppData\Local\Temp\ipykernel_18824\1458014979.py:123: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,2.414700
20,2.589900
30,2.256900
40,2.198600
50,2.040000


Saved fine-tuned model to: d:\elevvo-mine\summarizer_ft\final
ROUGE after fine-tuning: {'rouge1': 0.3651, 'rouge2': 0.1676, 'rougeL': 0.2575}
ROUGE after fine-tuning: {'rouge1': 0.3651, 'rouge2': 0.1676, 'rougeL': 0.2575}


## 9) Save Results

In [31]:

# Save the subset with generated summaries to a CSV for inspection
out_path = "summaries_output.csv"
subset.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: summaries_output.csv


## 10) Test Models using a Streamlit Interface

In [2]:

import os, time, json
from textwrap import dedent

# Use the model names already loaded in this notebook (fallback if empty)
try:
    _model_names = sorted({m[0] for m in MODELS})
except Exception:
    _model_names = []
if not _model_names:
    _model_names = [
        "ainize/bart-base-cnn",
        "google-t5/t5-small",
    ]

# Include local fine-tuned checkpoints if present
_local_paths = []
ft_root = "summarizer_ft"
final_dir = os.path.join(ft_root, "final")
if os.path.isdir(final_dir):
    _local_paths.append(final_dir)
# Also scan for Trainer checkpoints like summarizer_ft/checkpoint-*
if os.path.isdir(ft_root):
    for name in os.listdir(ft_root):
        p = os.path.join(ft_root, name)
        if os.path.isdir(p) and name.startswith("checkpoint-"):
            # Heuristic: only add if it looks like a HF checkpoint (config.json exists)
            if os.path.exists(os.path.join(p, "config.json")):
                _local_paths.append(p)

# Compose candidate list: local paths first, then hub model IDs
_candidate_list = _local_paths + _model_names

app_code = dedent(f"""
import os, re, time, io
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Optional parsers
try:
    import PyPDF2 as _pypdf2
except Exception:
    _pypdf2 = None
try:
    from docx import Document as _DocxDocument
except Exception:
    _DocxDocument = None

st.set_page_config(page_title="Summarizer", page_icon="📝", layout="wide")
st.title("📝 Summarizer App")

# Device selection
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
st.sidebar.markdown(f"**Device:** {{DEVICE}}")

CANDIDATE_MODELS = {json.dumps(_candidate_list)}

# Caching compatibility (Streamlit >=1.18 uses cache_resource)
def _apply_cache(fn):
    if hasattr(st, "cache_resource"):
        return st.cache_resource(show_spinner=True)(fn)
    # Fallback for older versions
    return st.cache(allow_output_mutation=True)(fn)

def load_model(name: str):
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(name).to(DEVICE)
    mdl.eval()
    return tok, mdl

load_model = _apply_cache(load_model)

# Safe sentence tokenize (no nltk dependency required at runtime)
def safe_sent_tokenize(text: str):
    return [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]

# Chunking and summarization helpers

def chunk_text(text: str, tokenizer, max_input_tokens: int = 1024, overlap_sentences: int = 1):
    sents = safe_sent_tokenize(text)
    chunks, buf = [], []
    for s in sents:
        candidate = " ".join(buf + [s])
        if len(tokenizer.encode(candidate, truncation=False)) <= max_input_tokens:
            buf.append(s)
        else:
            if buf:
                chunks.append(" ".join(buf))
                buf = buf[-overlap_sentences:] if overlap_sentences > 0 else []
                buf.append(s)
            else:
                tokens = tokenizer.encode(s, truncation=True, max_length=max_input_tokens)
                chunks.append(tokenizer.decode(tokens, skip_special_tokens=True))
                buf = []
    if buf:
        chunks.append(" ".join(buf))
    return chunks


def summarize_chunks(chunks, tokenizer, model, summary_max_tokens=128, num_beams=4):
    parts = []
    for ch in chunks:
        inputs = tokenizer(ch, return_tensors="pt", truncation=True, max_length=min(2048, tokenizer.model_max_length)).to(DEVICE)
        with torch.no_grad():
            ids = model.generate(**inputs, max_new_tokens=summary_max_tokens, num_beams=num_beams, early_stopping=True)
        parts.append(tokenizer.decode(ids[0], skip_special_tokens=True))
    return " ".join(parts)


def abstractive_summarize(text: str, tokenizer, model, max_input_tokens=1024, summary_max_tokens=128, use_chunking=True, num_beams=4, use_t5_prefix=False):
    if use_t5_prefix:
        text = "summarize: " + text
    tokens = tokenizer.encode(text, truncation=False)
    if len(tokens) <= max_input_tokens or not use_chunking:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_input_tokens).to(DEVICE)
        with torch.no_grad():
            ids = model.generate(**inputs, max_new_tokens=summary_max_tokens, num_beams=num_beams, early_stopping=True)
        return tokenizer.decode(ids[0], skip_special_tokens=True)
    chunks = chunk_text(text, tokenizer, max_input_tokens=max_input_tokens)
    return summarize_chunks(chunks, tokenizer, model, summary_max_tokens=summary_max_tokens, num_beams=num_beams)

# ---- File upload & extraction ----

def _decode_bytes(data: bytes) -> str:
    if not isinstance(data, (bytes, bytearray)):
        return str(data)
    try:
        return data.decode("utf-8")
    except Exception:
        try:
            return data.decode("latin-1")
        except Exception:
            return ""

def extract_text_from_upload(up_file) -> str:
    name = (up_file.name or "uploaded").lower()
    data = up_file.read()
    bio = io.BytesIO(data)
    if name.endswith(".txt"):
        return _decode_bytes(data)
    if name.endswith(".pdf"):
        if _pypdf2 is None:
            return ""
        try:
            reader = _pypdf2.PdfReader(bio)
            texts = []
            for pg in reader.pages:
                try:
                    t = pg.extract_text() or ""
                    if t:
                        texts.append(t)
                except Exception:
                    pass
            return " ".join(texts).strip()
        except Exception:
            return ""
    if name.endswith(".docx"):
        if _DocxDocument is None:
            return ""
        try:
            doc = _DocxDocument(bio)
            return " ".join([p.text for p in doc.paragraphs if p.text and p.text.strip()]).strip()
        except Exception:
            return ""
    return ""

# Sidebar controls
model_name = st.sidebar.selectbox("Model", options=CANDIDATE_MODELS, index=0)
max_input_tokens = st.sidebar.slider("Max input tokens", 256, 2048, 1024, 64)
summary_max_tokens = st.sidebar.slider("Summary max tokens", 16, 256, 128, 8)
num_beams = st.sidebar.slider("Beam size", 1, 6, 4, 1)
use_chunking = st.sidebar.checkbox("Use chunking for long texts", value=True)
use_t5_prefix = "t5" in model_name.lower()

# Visual cue if a local fine-tuned model is selected
if os.path.isdir(model_name):
    st.sidebar.success("Using local fine-tuned model")

st.sidebar.caption("Tip: T5 models automatically use the 'summarize:' prefix.")

# Backward-compatible primary button

def primary_button(label: str):
    try:
        return st.button(label, type="primary")
    except TypeError:
        return st.button(label)

# File uploader
uploaded = st.file_uploader("Upload PDF, DOCX, or TXT", type=["pdf","docx","txt"], accept_multiple_files=True)
combined_text = ""
if uploaded:
    texts = []
    for f in uploaded:
        t = extract_text_from_upload(f)
        if t:
            texts.append(t)
        else:
            nm = f.name or "file"
            if nm.lower().endswith(".pdf") and _pypdf2 is None:
                st.warning("Couldn't read " + nm + ". Install PyPDF2 to enable PDF parsing: pip install PyPDF2")
            elif nm.lower().endswith(".docx") and _DocxDocument is None:
                st.warning("Couldn't read " + nm + ". Install python-docx to enable DOCX parsing: pip install python-docx")
            else:
                st.warning("Couldn't extract text from " + nm)
    if texts:
        combined_text = "  ".join(texts)
        st.success("Loaded " + str(len(texts)) + " file(s)")
        if not st.session_state.get("input_text"):
            st.session_state["input_text"] = combined_text[:200000]

# Main input area
text = st.text_area("Enter article text", height=300, placeholder="Paste a long article here…", key="input_text")

if primary_button("Summarize"):
    if not text or not text.strip():
        st.warning("Please paste some text or upload a file.")
    else:
        with st.spinner("Loading model and generating…"):
            tok, mdl = load_model(model_name)
            t0 = time.time()
            out = abstractive_summarize(
                text.strip(), tok, mdl,
                max_input_tokens=max_input_tokens,
                summary_max_tokens=summary_max_tokens,
                use_chunking=use_chunking,
                num_beams=num_beams,
                use_t5_prefix=use_t5_prefix,
            )
            dt = time.time() - t0
        st.success(f"Done in {{dt:.2f}}s")
        st.subheader("Summary")
        st.write(out)
""")

out_path = os.path.join(os.getcwd(), "summarizer_app.py")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(app_code)

print("Wrote:", out_path)
print("To launch the app from PowerShell:")
print("\n  streamlit run summarizer_app.py\n")


Wrote: d:\elevvo-mine\summarizer_app.py
To launch the app from PowerShell:

  streamlit run summarizer_app.py



<>:234: SyntaxWarning: invalid escape sequence '\s'
<>:234: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Youssef\AppData\Local\Temp\ipykernel_12952\3243301361.py:234: SyntaxWarning: invalid escape sequence '\s'
  """)



## 11) Summary
- Used **pre-trained encoder–decoder models** (BART/T5/Pegasus) for **abstractive** summarization.
- Implemented **long-text handling** via truncation and a **chunk-and-merge** strategy.
- Evaluated with **ROUGE-1/2/L** (F1) using `rouge-score`.
- Implemented **extractive TextRank** and compared outputs qualitatively.
- Included a **fine-tuning** template with `Trainer` for your custom dataset.

**Next steps:**
- Tune generation params (`num_beams`, `min_length`, `length_penalty`) per model.
- Add coverage penalty or nucleus sampling for diverse summaries.
- For long documents, summarize-by-sections then re-summarize the summaries.
- Use `enforce_max_length=False` and careful chunking for models with smaller context.
